In [1]:
import pandas as pd
import numpy as np

import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_curve

from scipy.stats import pearsonr, chi2_contingency
from itertools import combinations
from statsmodels.stats.proportion import proportion_confint

# 1. Introduction

The presented analysis was inspired by the second chapter of [this kaggle notebook](https://www.kaggle.com/code/pavlofesenko/minimizing-risks-for-loan-investments). Some changes were made but the main ideas and most of the explanations are taken from this solution.

# 2. Data preprocessing

In [2]:
data = pd.read_csv(
    '../data/raw/accepted_2007_to_2018Q4.csv',
    parse_dates=['issue_d'], infer_datetime_format=True)
data = data.reset_index(drop=True)
data.head()

d:\Users\Michal\Documents\Studia\semestr2\NLP\nlp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## 2.1. Target column encoding

First, I encode the target column according to the specification.

In [3]:
X = data[data['loan_status'] != 'Current'].copy()
X.loc[X['loan_status'] == 'Fully Paid', 'target'] = 0
X.loc[X['loan_status'] != 'Fully Paid', 'target'] = 1
X = X.drop(columns=['id', 'loan_status'])

In [4]:
X.target.value_counts()

0.0    1076751
1.0     305633
Name: target, dtype: int64

## 2.2. Feature types

Let's check the categorical features and see if any of them could be transformed to other types.

In [5]:
X.select_dtypes('object').head()

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,pymnt_plan,url,desc,...,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
0,36 months,C,C4,leadman,10+ years,MORTGAGE,Not Verified,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
1,36 months,C,C1,Engineer,10+ years,MORTGAGE,Not Verified,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
2,60 months,B,B4,truck driver,10+ years,MORTGAGE,Not Verified,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
4,60 months,F,F1,Contract Specialist,3 years,MORTGAGE,Source Verified,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
5,36 months,C,C3,Veterinary Tecnician,4 years,RENT,Source Verified,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,...,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN


The features `earliest_cr_line` and `sec_app_earliest_cr_line` are dates and their type should be changed to `datetime`. Later they will be transformed to ordinal numeric features by the machine learning model.

In [6]:
X['earliest_cr_line'] = pd.to_datetime(X['earliest_cr_line'], infer_datetime_format=True)
X['last_credit_pull_d'] = pd.to_datetime(X['last_credit_pull_d'], infer_datetime_format=True)
X['last_pymnt_d'] = pd.to_datetime(X['last_pymnt_d'], infer_datetime_format=True)
X['sec_app_earliest_cr_line'] = pd.to_datetime(X['sec_app_earliest_cr_line'], infer_datetime_format=True)

The features `emp_length` and `id` are numeric and their type should be changed to `float`. In case of `emp_length` I replace the extreme cases of "< 1 year" and "10+ years" with "0 years" and "11 years" respectively to separate these groups from the rest.

In [7]:
X['emp_length'] = X['emp_length'].replace({'< 1 year': '0 years', '10+ years': '11 years'})
X['emp_length'] = X['emp_length'].str.extract('(\d+)').astype('float')

Then the data is split into the training and testing part. All analysis will be performed on the training dataset in order to prevent any data leakage.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
        X.drop(columns=['target']), X['target'], test_size=0.2, stratify=X['target'],
        random_state=42
    )
X_train['target'] = y_train
X_test['target'] = y_test

## 2.3. Missing values

Although the machine learning model used here can deal with missing values, it is a good practice to do it yourself or ideally with a domain expert. Below is the table of columns with missing values and their ratio to the total number of rows.

In [9]:
nan_mean = X_train.isna().mean()
nan_mean = nan_mean[nan_mean != 0].sort_values()
nan_mean[nan_mean > 0.9]

desc                                          0.908937
next_pymnt_d                                  0.973148
debt_settlement_flag_date                     0.975247
settlement_date                               0.975247
settlement_amount                             0.975247
settlement_status                             0.975247
settlement_term                               0.975247
settlement_percentage                         0.975247
annual_inc_joint                              0.978502
dti_joint                                     0.978505
verification_status_joint                     0.978690
sec_app_inq_last_6mths                        0.983896
sec_app_fico_range_low                        0.983896
sec_app_earliest_cr_line                      0.983896
sec_app_fico_range_high                       0.983896
sec_app_chargeoff_within_12_mths              0.983896
sec_app_num_rev_accts                         0.983896
sec_app_open_act_il                           0.983896
sec_app_op

In [10]:
X_train = X_train.drop(nan_mean[nan_mean > 0.9].keys(), axis=1, errors='ignore')
X_test = X_test.drop(nan_mean[nan_mean > 0.9].keys(), axis=1, errors='ignore')

For categorical features `emp_title` the missing values should be filled with an empty string so these features are placed in the list `fill_empty`.

For some of the numeric features the missing values should be filled using the maximum value of the respective columns so these features are placed in the list `fill_max`. For example, the feature `mths_since_last_record` indicates the number of months since the last record (like bankruptcy, foreclosure, tax liens, etc.) so if missing, one should assume that no records were made and the number of months since the "last" record should be a maximum.

For the rest of the numeric features the missing values should be filled using the minimum value of the respective columns so these features are placed in the list `fill_min`. For example, the feature `emp_length` indicates the number of working years so if missing, one should assume that the borrower never worked and the number of working years should be a minimum.

The rest of the categorical columns are filled using the most frequent values.

In [11]:
fill_empty = ['emp_title']
fill_max = ['bc_open_to_buy', 'mo_sin_old_il_acct', 'mths_since_last_delinq',
            'mths_since_last_major_derog', 'mths_since_last_record',
            'mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
            'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
            'pct_tl_nvr_dlq']
fill_min = np.setdiff1d(X_train.columns.values, np.append(fill_empty, fill_max))

X_train[fill_empty] = X_train[fill_empty].fillna('')
X_test[fill_empty] = X_test[fill_empty].fillna('')
X_train[fill_max] = X_train[fill_max].fillna(X_train[fill_max].max(numeric_only=True))
X_test[fill_max] = X_test[fill_max].fillna(X_train[fill_max].max(numeric_only=True))
X_train[fill_min] = X_train[fill_min].fillna(X_train[fill_min].min(numeric_only=True))
X_test[fill_min] = X_test[fill_min].fillna(X_train[fill_min].min(numeric_only=True))

fill_most_frequent = np.setdiff1d(X_train.select_dtypes('object').columns.values, fill_empty)
X_train[fill_most_frequent] = X_train[fill_most_frequent].fillna(X_train[fill_most_frequent].mode().iloc[0])
X_test[fill_most_frequent] = X_test[fill_most_frequent].fillna(X_train[fill_most_frequent].mode().iloc[0])

## 2.3. Multicollinearity

Although highly correlated features (*multicollinearity*) aren't a problem for the machine learning models based on decision trees (as used here), these features decrease importances of each other and can make feature analysis more difficult. Therefore, I calculate feature correlations and remove the features with very high correlation coefficients before applying machine learning.

I start with numeric features and before calculating their correlations, it's a good practice to look at the number of their unique values.

In [12]:
num_feat = X_train.select_dtypes('number').columns.values
X_train[num_feat].nunique().sort_values()

policy_code               1
target                    2
num_tl_30dpd              5
num_tl_120dpd_2m          6
acc_now_delinq            8
                     ...   
tot_hi_cred_lim      392216
total_rec_int        475261
last_pymnt_amnt      590210
total_pymnt_inv      863846
total_pymnt         1041761
Length: 90, dtype: int64

In [13]:
X_train = X_train.drop(['policy_code'], axis=1, errors='ignore')
X_test = X_test.drop(['policy_code'], axis=1, errors='ignore')

For all pairs of the numeric features `comb_num_feat` I calculate their Pearson's R correlation coefficient and store it in `corr_num_feat`.

In [14]:
num_feat = X_train.select_dtypes('number').columns.values
comb_num_feat = np.array(list(combinations(num_feat, 2)))
corr_num_feat = np.array([])
for comb in comb_num_feat:
    corr = pearsonr(X_train[comb[0]], X_train[comb[1]])[0]
    corr_num_feat = np.append(corr_num_feat, corr)

The highly correlated pairs with the absolute value of their correlation coefficient ≥0.9 are printed below.

In [15]:
high_corr_num = comb_num_feat[np.abs(corr_num_feat) >= 0.9]
high_corr_num

array([['loan_amnt', 'funded_amnt'],
       ['loan_amnt', 'funded_amnt_inv'],
       ['loan_amnt', 'installment'],
       ['funded_amnt', 'funded_amnt_inv'],
       ['funded_amnt', 'installment'],
       ['funded_amnt_inv', 'installment'],
       ['fico_range_low', 'fico_range_high'],
       ['open_acc', 'num_sats'],
       ['out_prncp', 'out_prncp_inv'],
       ['total_pymnt', 'total_pymnt_inv'],
       ['total_pymnt', 'total_rec_prncp'],
       ['total_pymnt_inv', 'total_rec_prncp'],
       ['recoveries', 'collection_recovery_fee'],
       ['tot_cur_bal', 'tot_hi_cred_lim'],
       ['bc_open_to_buy', 'mths_since_recent_bc'],
       ['num_actv_rev_tl', 'num_rev_tl_bal_gt_0']], dtype='<U30')

The first feature (chosen arbitrarily) from each highly correlated feature pair is then removed.

In [16]:
X_train = X_train.drop(np.unique(high_corr_num[:, 0]), axis=1, errors='ignore')
X_test = X_test.drop(np.unique(high_corr_num[:, 0]), axis=1, errors='ignore')

Then I print out the number of unique values for categorical features.

In [17]:
cat_feat = X_train.select_dtypes('object').columns.values
X_train[cat_feat].nunique(dropna=False).sort_values()

term                          2
hardship_flag                 2
application_type              2
initial_list_status           2
disbursement_method           2
pymnt_plan                    2
debt_settlement_flag          2
verification_status           3
home_ownership                6
grade                         7
purpose                      14
sub_grade                    35
addr_state                   51
zip_code                    933
title                     52151
emp_title                322599
url                     1105880
dtype: int64

The feature `url` has a unique value for each entry and should be removed to avoid overfitting. The feature `emp_title` has very large number of unique values and leads to the memory error when creating a contingency table, therefore I remove it as well. I remove also the feature `debt_settlement_flag` since it's available only after the loan is settled.

In [18]:
X_train = X_train.drop(['url', 'emp_title', 'debt_settlement_flag'], axis=1, errors='ignore')
X_test = X_test.drop(['url', 'emp_title', 'debt_settlement_flag'], axis=1, errors='ignore')

For all pairs of the categorical features `comb_cat_feat` I calculate the Cramer's V correlation coefficient that is expressed through the chi-square statistic $\chi^2$ of the contingency table:

$$ V = \sqrt{ \frac{ \chi^2 }{ n (\text{min}(K_1, K_2) - 1) } } $$

where $n$ is the sum of all elements in the contingency table, $K_1$ and $K_2$ are the dimensions of the contingency table. Note that Pearson's R correlation coefficient isn't applicable to categorical features and shouldn't be used.

In [19]:
cat_feat = X_train.select_dtypes('object').columns.values
comb_cat_feat = np.array(list(combinations(cat_feat, 2)))
corr_cat_feat = np.array([])
for comb in comb_cat_feat:
    table = pd.pivot_table(X_train, values='target', index=comb[0], columns=comb[1], aggfunc='count').fillna(0)
    corr = np.sqrt(chi2_contingency(table)[0] / (table.values.sum() * (np.min(table.shape) - 1) ) )
    corr_cat_feat = np.append(corr_cat_feat, corr)

The highly correlated pairs with the absolute value of their correlation coefficient ≥0.9 are printed below.

In [20]:
high_corr_cat = comb_cat_feat[corr_cat_feat >= 0.9]
high_corr_cat

array([['grade', 'sub_grade'],
       ['purpose', 'title'],
       ['zip_code', 'addr_state']], dtype='<U19')

This time I remove the feature with more unique values from each highly correlated feature pair in order to reduce the number of options.

In [21]:
cat_to_drop = ['sub_grade', 'title', 'zip_code']
X_train = X_train.drop(columns=cat_to_drop, axis=1)
X_test = X_test.drop(columns=cat_to_drop, axis=1)

Lastly I encode all categorical features using the one-hot method, which leaves us with 173 features.

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

oh_columns = X_train.select_dtypes('object').columns.values
column_trans = ColumnTransformer(
    [('categories', OneHotEncoder(dtype='int', sparse=False), oh_columns)],
    remainder='passthrough', verbose_feature_names_out=False)

X_train_encoded = column_trans.fit_transform(X_train)
X_test_encoded = column_trans.transform(X_test)
X_train_encoded.shape

(1105907, 174)